Here is first step to convert review file to csv

In [2]:
import csv
import re
import requests

# Specify the input and output file paths
input_url = 'https://github.com/Vovanch0/sentiment-analysis-isy503/raw/main/data/dvd_positive.review'
output_file = 'dvd_positive.csv'

# Download the content of the .review file from the URL
response = requests.get(input_url)
data = response.text


# Split the data into individual reviews
reviews = re.findall(r'<review>(.*?)</review>', data, re.DOTALL)

# Prepare the CSV file headers
headers = [
    'unique_id', 'asin', 'product_name', 'product_type', 'helpful',
    'rating', 'title', 'date', 'reviewer', 'reviewer_location', 'review_text'
]


# Open a CSV file for writing
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()

    # Iterate through each review and extract the fields
    for review in reviews:
        review_dict = {}
        for header in headers:
            pattern = f'<{header}>(.*?)</{header}>'
            match = re.search(pattern, review, re.DOTALL)
            review_dict[header] = match.group(1).strip() if match else ''

        # Write the row to the CSV
        writer.writerow(review_dict)

print(f"CSV file '{output_file}' created successfully.")

CSV file 'dvdpos.csv' created successfully.


Local save from virtual Google colab fs

In [ ]:
from google.colab import files
files.download('dvd_positive.csv')

Merge all csv file to one csv

In [ ]:
import os
import pandas as pd

def merge_csv_files(folder_path, output_file):
    # List to hold all the dataframes
    all_dataframes = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            csv_file = os.path.join(folder_path, filename)
            # Read the CSV file into a DataFrame
            df = pd.read_csv(csv_file)
            all_dataframes.append(df)

    # Concatenate all DataFrames
    merged_df = pd.concat(all_dataframes, ignore_index=True)

    # Write the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)

folder_path = "C:/503/final"  # Replace with your folder path
output_file = "C:/503/final/merged_output.csv"    # Name of the output file

merge_csv_files(folder_path, output_file)

After that, I have uploaded this file to github.

Link
https://github.com/Vovanch0/sentiment-analysis-isy503/raw/main/data/merged_output.csv


In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

import pandas as pd
data = pd.read_csv('https://github.com/Vovanch0/sentiment-analysis-isy503/raw/main/data/merged_output.csv',
                   sep=',', encoding='utf8')



In [33]:
print('Number of reviews', str(len(data)))

Number of reviews 8000


In [ ]:
data.head(15)

Encoding the words

Encode the labels for ‘positive’ and ‘negative’

In [30]:
# Create a new column called 'type'.
# If the rating is greater than 3, then the type of review is positive (1),
# otherwise it is negative (0)
data['type'] = np.where(data['rating'] > 3, 1, 0)

# Delete the unnecessary columns
data.drop(['unique_id', 'asin', 'product_name', 'product_type', 'helpful',
           'date', 'reviewer', 'reviewer_location'], axis=1, inplace=True)

# Show the number of positive and negative reviews
type_review = data['type'].value_counts()
print("Number of positive reviews:", type_review[1])
print("Number of negative reviews:", type_review[0])

Number of positive reviews: 4000
Number of negative reviews: 4000


In [32]:
data.head(15)

,rating,title,review_text,type
0,1.0,"Horrible book, horrible.","THis book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the...",0
1,1.0,shallow self-indulgence,"I like to use the Amazon reviews when purchasing books, especially alert for dissenting perceptions about higly rated items, which usually disuades me from a selection. So I offer this review that seriously questions the popularity of this work ...",0
2,1.0,"Horrible book, horrible.","THis book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the...",0
3,1.0,Disappointment,"I'm not sure who's writing these reviews, but I read repitition after repitition after repitition, with a lot of confusing fluff. Asking hard questions to solve complex or simple issues. What's ""fierce"" about that? \n\nThe writing style and conte...",0
4,2.0,A Disappointing Mess,"I picked up the first book in this series (The Eyre Affair) based purely on its premise and was left somewhat underwhelmed. Still, the potential for the series seemed so large that I went ahead and read this second one too, only to be even less e...",0
5,2.0,"Save your money, there are better books out there","Not only do I disagree with his opinions, but some of his facts are blatently false. On page 106 he refers to violence agains Native Americans by European settlers as ""isolated instances"" and not planned nor calculated. Had to read it for a cla...",0
6,1.0,"Thank you, but no, thanks","&quot;I have received your new book against the human race, and thank you for it. Never was such cleverness used in the design of making us all stupid. One longs, in reading your book, to walk on all fours. But as I have lost that habit for more ...",0
7,1.0,Unendurable,"This book was on somebody's Amazon.com LISTMANIA for Sea Turtles, so I hunted it down. It is only vaguely about sea turtles, in that it is set on an island in Georgia where sea turtles nest. There is a sea turtle biologist introduced at the ver...",0
8,1.0,The Hard Way,"I am not sure whatever possessed me to buy this book. Honestly, it was a complete waste of my time. To quote a friend, it was not the best use of my entertainment dollar. If you are a fan of pedestrian writing, lack-luster plots and hackneyed ...",0
9,2.0,Some good info among the political commercial,"When Professor Polk describes the sweep of history in Iraq, he is at his best, but the book often descends into a political diatribe that is neither appropriate nor grounded in the facts. Much of the political ranting is supposition, marring wha...",0


Let's see unique values in the 'rating'

In [31]:
data['rating'].unique()

array([1., 2., 5., 4.])

**Clean data | Speller**


1 Initialize the autocorrect object for English

2 Apply the autocorrect function to each row in the review_text column, and stores the result in a new column review_text_correct.

3 Save the cleaned data to a new CSV file cleaned_reviews.csv

Proccessing time - around 55 minutes!!!

In [34]:
!pip install autocorrect
from autocorrect import Speller

# Set language to Eng
spell = Speller(lang='en')

# Apply autocorrect
data['review_text_correct'] = data['review_text'].apply(spell)

# Save the cleaned data to a CSV file
output_file = 'cleaned_reviews.csv'
data.to_csv(output_file, index=False, encoding='utf-8')

print(f"Cleaned data saved to '{output_file}'")

Cleaned data saved to 'cleaned_reviews.csv'


In [35]:
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.options.display.max_colwidth = 250

data.loc[7, ['review_text', 'review_text_correct']]

As the processing time is too long, so I uploaded this processed file to Github

In [40]:
data = pd.read_csv('https://github.com/Vovanch0/sentiment-analysis-isy503/raw/main/data/cleaned_reviews.csv')

data.head(3)

,rating,title,review_text,type,review_text_correct
0,1.0,"Horrible book, horrible.","THis book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the...",0,"This book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the..."
1,1.0,shallow self-indulgence,"I like to use the Amazon reviews when purchasing books, especially alert for dissenting perceptions about higly rated items, which usually disuades me from a selection. So I offer this review that seriously questions the popularity of this work ...",0,"I like to use the Amazon reviews when purchasing books, especially alert for dissenting perceptions about highly rated items, which usually diseases me from a selection. So I offer this review that seriously questions the popularity of this work..."
2,1.0,"Horrible book, horrible.","THis book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the...",0,"This book was horrible. If it was possible to rate it lower than one star i would have. I am an avid reader and picked this book up after my mom had gotten it from a friend. I read half of it, suffering from a headache the entire time, and the..."
